# Підготовка та аналіз даних 
## Лабораторна робота №1
### Наука про дані: підготовчий етап
### ФБ-21 Каюн Вероніка
<b>Мета роботи:</b> ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції


In [4]:
import os
import pandas as pd
import urllib.request
import datetime
import shutil
print("Setup ok")

Setup ok


• Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [5]:
directory='files'
if os.path.exists(directory):
    shutil.rmtree(directory)
os.makedirs(directory, exist_ok=True)
def download_data(province_id,year1=1981,year2=2024):
        
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1={year1}&year2={year2}&type=Mean"
    vhi_url = urllib.request.urlopen(url)
    current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    filename = f'vhi_id__{province_id}__{current_datetime}'
    
    out = open(f'files/{filename}.csv','wb')
    out.write(vhi_url.read())
    out.close()
    print(f"VHI is downloaded in {filename}")
    return filename



In [6]:
# Завантажуємо дані для кожної адміністративної одиниці
for i in range(1, 28):
    download_data(i)

VHI is downloaded in vhi_id__1__2024-03-22_20-52
VHI is downloaded in vhi_id__2__2024-03-22_20-52
VHI is downloaded in vhi_id__3__2024-03-22_20-52
VHI is downloaded in vhi_id__4__2024-03-22_20-52
VHI is downloaded in vhi_id__5__2024-03-22_20-52
VHI is downloaded in vhi_id__6__2024-03-22_20-52
VHI is downloaded in vhi_id__7__2024-03-22_20-52
VHI is downloaded in vhi_id__8__2024-03-22_20-52
VHI is downloaded in vhi_id__9__2024-03-22_20-52
VHI is downloaded in vhi_id__10__2024-03-22_20-52
VHI is downloaded in vhi_id__11__2024-03-22_20-52
VHI is downloaded in vhi_id__12__2024-03-22_20-52
VHI is downloaded in vhi_id__13__2024-03-22_20-52
VHI is downloaded in vhi_id__14__2024-03-22_20-52
VHI is downloaded in vhi_id__15__2024-03-22_20-52
VHI is downloaded in vhi_id__16__2024-03-22_20-52
VHI is downloaded in vhi_id__17__2024-03-22_20-52
VHI is downloaded in vhi_id__18__2024-03-22_20-52
VHI is downloaded in vhi_id__19__2024-03-22_20-52
VHI is downloaded in vhi_id__20__2024-03-22_20-52
VHI is do

• Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [20]:
def read_files(directory):
  files = os.listdir(directory)
  headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
  data_frame = pd.DataFrame()
  for i in range(len(files)):
    file_path = os.path.join(directory, files[i])
    df = pd.read_csv(file_path, header = 1, names = headers)
    df = df.drop('empty', axis=1)
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df['area'] = i+1
    df['Year'] = df['Year'].str.replace('<tt><pre>', '')
    df = df.drop(df[df['Year'] == '</pre></tt>'].index)
    df["Year"] = df["Year"].astype(int)
    data_frame = pd.concat([data_frame, df])
    
  data_frame=data_frame.reset_index(drop=True) 
  return data_frame

In [21]:
df = read_files(directory)
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95     1
1      1982   2.0  0.063  261.53  55.89  38.20  47.04     1
2      1982   3.0  0.063  263.45  57.30  32.69  44.99     1
3      1982   4.0  0.061  265.10  53.96  28.62  41.29     1
4      1982   5.0  0.058  266.42  46.87  28.57  37.72     1
...     ...   ...    ...     ...    ...    ...    ...   ...
57910  2024   7.0  0.121  279.49  61.95   4.74  33.34    27
57911  2024   8.0  0.132  281.65  61.16   4.58  32.87    27
57912  2024   9.0  0.143  283.52  60.00   5.64  32.82    27
57913  2024  10.0  0.152  285.16  57.23   8.11  32.67    27
57914  2024  11.0  0.153  285.44  52.03  14.62  33.33    27

[57915 rows x 8 columns]


• Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні:

In [22]:
def rename_region(df):
  names =  {1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька", 5: "Житомирська", 6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська", 9: "Київська", 10: "Кіровоградська", 11: "Луганська", 12: "Львівська", 13: "Миколаївська", 14: "Одеська", 15: "Полтавська", 16: "Рівенська", 17: "Сумська", 18: "Тернопільська", 19: "Харківська", 20: "Херсонська", 21: "Хмельницька", 22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська", 25: "Республіка Крим", 26: "Київ", 27: "Севастопіль"}
 
  for name in names:
    df["area"] = df["area"].replace({name:names[name]})
  return df

In [23]:
df = rename_region(read_files(directory))
print(df)


       Year  Week    SMN     SMT    VCI    TCI    VHI         area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95    Вінницька
1      1982   2.0  0.063  261.53  55.89  38.20  47.04    Вінницька
2      1982   3.0  0.063  263.45  57.30  32.69  44.99    Вінницька
3      1982   4.0  0.061  265.10  53.96  28.62  41.29    Вінницька
4      1982   5.0  0.058  266.42  46.87  28.57  37.72    Вінницька
...     ...   ...    ...     ...    ...    ...    ...          ...
57910  2024   7.0  0.121  279.49  61.95   4.74  33.34  Севастопіль
57911  2024   8.0  0.132  281.65  61.16   4.58  32.87  Севастопіль
57912  2024   9.0  0.143  283.52  60.00   5.64  32.82  Севастопіль
57913  2024  10.0  0.152  285.16  57.23   8.11  32.67  Севастопіль
57914  2024  11.0  0.153  285.44  52.03  14.62  33.33  Севастопіль

[57915 rows x 8 columns]


Ряд VHI для області за рік та пошук екстремумів (мінімуму та максимуму):

In [11]:
def find_extremes(df, area, year):
    region_year_data = df[(df["area"] == area) & (df["Year"] == year)]
    vhi_series = region_year_data["VHI"]
    
    min_vhi = vhi_series.min()
    max_vhi = vhi_series.max()
    
    return {'min': min_vhi, 'max': max_vhi}

def extremes_for_regions(df, year):
    regions = df["area"].unique()
    for region in regions:
        extremes = find_extremes(df, region, year)
        print(f"Екстремуми для області {region} у {year} році:")
        print(f"Мінімум: {extremes['min']}")
        print(f"Максимум: {extremes['max']}\n")

# Приклад використання:
year = 2020  
extremes_for_regions(df, year)




Екстремуми для області Вінницька у 2020 році:
Мінімум: 37.43
Максимум: 67.65

Екстремуми для області Волинська у 2020 році:
Мінімум: 29.27
Максимум: 56.3

Екстремуми для області Дніпропетровська у 2020 році:
Мінімум: 30.71
Максимум: 60.83

Екстремуми для області Донецька у 2020 році:
Мінімум: 22.73
Максимум: 63.63

Екстремуми для області Житомирська у 2020 році:
Мінімум: 21.29
Максимум: 64.39

Екстремуми для області Закарпатська у 2020 році:
Мінімум: 41.35
Максимум: 63.06

Екстремуми для області Запорізька у 2020 році:
Мінімум: 19.28
Максимум: 58.11

Екстремуми для області Івано-Франківська у 2020 році:
Мінімум: 22.77
Максимум: 46.71

Екстремуми для області Київська у 2020 році:
Мінімум: 26.22
Максимум: 64.77

Екстремуми для області Кіровоградська у 2020 році:
Мінімум: 37.68
Максимум: 66.01

Екстремуми для області Луганська у 2020 році:
Мінімум: 29.29
Максимум: 60.9

Екстремуми для області Львівська у 2020 році:
Мінімум: 36.76
Максимум: 51.22

Екстремуми для області Миколаївська у 2020

Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);

In [24]:
def find_drough(df, percent):
    result = []

    for year, group_df in df.groupby('Year'):
        total_areas = len(group_df['area'].unique())
        drought_areas = len(group_df[group_df['VHI'] < 15]['area'].unique())

        if (drought_areas / total_areas) > (percent / 100):
            percent_str = f"{round((drought_areas / total_areas) * 100, 2)}%"
            result.append((year, drought_areas, percent_str))


    return result

percent = 10
drought = find_drough(df, percent)
print(drought)

[(2000, 6, '22.22%'), (2007, 5, '18.52%')]


In [15]:
def vhi(df, year1, year2, areas):
    data = df[(df['Year']>=year1) & (df['Year']<=year2) & (df['area'].isin(areas))]
    return data
year1= 2015   
year2 =2020
areas=['Чернівецька', 'Вінницька']
vhi1=vhi(df, year1, year2, areas)
 
print(vhi1)

       index  Year  Week    SMN     SMT    VCI    TCI    VHI         area
1666    1716  2015   1.0  0.057  256.27  45.91  50.34  48.12    Вінницька
1667    1717  2015   2.0  0.059  255.97  50.00  52.62  51.31    Вінницька
1668    1718  2015   3.0  0.065  257.98  57.38  46.72  52.05    Вінницька
1669    1719  2015   4.0  0.070  260.68  61.66  39.14  50.40    Вінницька
1670    1720  2015   5.0  0.075  264.23  60.60  33.08  46.84    Вінницька
...      ...   ...   ...    ...     ...    ...    ...    ...          ...
49163   2023  2020  48.0  0.131  273.87  73.94  10.30  42.12  Чернівецька
49164   2024  2020  49.0  0.114  271.41  73.50  12.52  43.01  Чернівецька
49165   2025  2020  50.0  0.099  268.84  71.77  15.44  43.61  Чернівецька
49166   2026  2020  51.0  0.087  266.70  71.94  17.96  44.95  Чернівецька
49167   2027  2020  52.0  0.078  264.99  73.06  21.18  47.12  Чернівецька

[624 rows x 9 columns]


Аналогічно для помірних посух

In [25]:
def find_drough(df, percent):
    result = []

    for year, group_df in df.groupby('Year'):
        total_areas = len(group_df['area'].unique())
        drought_areas = len(group_df[(group_df['VHI'] > 15) & (group_df['VHI'] < 35)]['area'].unique())

        if (drought_areas / total_areas) > (percent / 100):
            percent_str = f"{round((drought_areas / total_areas) * 100, 2)}%"
            result.append((year, drought_areas, percent_str))

    return result

percent = 10
drought = find_drough(df, percent)
print(drought)


[(1982, 27, '100.0%'), (1983, 26, '96.3%'), (1984, 26, '96.3%'), (1985, 26, '96.3%'), (1986, 27, '100.0%'), (1987, 20, '74.07%'), (1988, 24, '88.89%'), (1989, 26, '96.3%'), (1990, 25, '92.59%'), (1991, 24, '88.89%'), (1992, 24, '88.89%'), (1993, 25, '92.59%'), (1994, 22, '81.48%'), (1995, 26, '96.3%'), (1996, 26, '96.3%'), (1997, 16, '59.26%'), (1998, 7, '25.93%'), (1999, 22, '81.48%'), (2000, 27, '100.0%'), (2001, 12, '44.44%'), (2002, 12, '44.44%'), (2003, 13, '48.15%'), (2004, 3, '11.11%'), (2005, 7, '25.93%'), (2006, 13, '48.15%'), (2007, 15, '55.56%'), (2008, 16, '59.26%'), (2009, 16, '59.26%'), (2010, 14, '51.85%'), (2011, 18, '66.67%'), (2012, 13, '48.15%'), (2013, 17, '62.96%'), (2014, 11, '40.74%'), (2015, 24, '88.89%'), (2016, 15, '55.56%'), (2017, 11, '40.74%'), (2018, 15, '55.56%'), (2019, 26, '96.3%'), (2020, 17, '62.96%'), (2021, 11, '40.74%'), (2022, 7, '25.93%'), (2023, 21, '77.78%')]
